In [73]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import betfairlightweight
from betfairlightweight import filters
import ast

In [127]:
# Load environment variables
load_dotenv()

# Betfair API setup
bf_usr = os.getenv("BF_LOGIN")
bf_pass = os.getenv("BF_PASS")
bf_api = os.getenv("BF_API_KEY")
bf_certs_path = '../certs/'

# Initialize Betfair client
client = betfairlightweight.APIClient(bf_usr, bf_pass, app_key=bf_api, certs=bf_certs_path)
client.login()

<LoginResource>

In [30]:
def get_polymarket_data():
    r = requests.get("https://gamma-api.polymarket.com/events?limit=200&closed=false")
    response = r.json()

    market_data = []
    for market in response:
        overall_data = {
            'Market ID': market.get('id', 'N/A'),
            'Title': market.get('title', 'N/A'),
            'End Date': market.get('endDate', 'N/A'),
            'Overall Liquidity': float(market.get('liquidity', 0)),
            'Overall Volume': float(market.get('volume', 0)),
            'Overall Volume 24hr': float(market.get('volume24hr', 0)),
        }
        for bet in market.get('markets', []):
            outcome_prices_str = bet.get('outcomePrices', '["N/A", "N/A"]')
            try:
                outcome_prices = ast.literal_eval(outcome_prices_str)
                yes_price = float(outcome_prices[0]) if len(outcome_prices) > 0 else None
                no_price = float(outcome_prices[1]) if len(outcome_prices) > 1 else None
            except (ValueError, SyntaxError):
                yes_price = None
                no_price = None
                
            bet_data = {
                'Bet ID': bet.get('id'),
                'Bet Question': bet.get('question', 'N/A'),
                'Bet Liquidity': float(bet.get('liquidity', 0)),
                'Bet Volume': float(bet.get('volume', 0)),
                'Bet Volume 24hr': float(bet.get('volume24hr', 0)),
                'Yes Price': yes_price,
                'No Price': no_price,
            }
            combined_data = {**overall_data, **bet_data}
            market_data.append(combined_data)

    return pd.DataFrame(market_data)

def get_betfair_data():
    market_filter = filters.market_filter(event_type_ids=['2378961'])  # Politics event type
    market_catalogues = client.betting.list_market_catalogue(
        filter=market_filter,
        max_results=100,
        market_projection=['RUNNER_DESCRIPTION']
    )

    market_ids = [market.market_id for market in market_catalogues]
    price_filter = filters.price_projection(price_data=['EX_BEST_OFFERS'])

    betfair_data = []
    for market in market_catalogues:
        market_books = client.betting.list_market_book(
            market_ids=[market.market_id],
            price_projection=price_filter
        )
        if market_books:
            market_book = market_books[0]
            for runner in market_book.runners:
                runner_data = {
                    'Market ID': market.market_id,
                    'Market Name': market.market_name,
                    'Selection ID': runner.selection_id,
                    'Selection Name': next((r.runner_name for r in market.runners if r.selection_id == runner.selection_id), 'Unknown'),
                    'Best Back Price': runner.ex.available_to_back[0].price if runner.ex.available_to_back else None,
                    'Best Back Size': runner.ex.available_to_back[0].size if runner.ex.available_to_back else None,
                    'Best Lay Price': runner.ex.available_to_lay[0].price if runner.ex.available_to_lay else None,
                    'Best Lay Size': runner.ex.available_to_lay[0].size if runner.ex.available_to_lay else None,
                }
                betfair_data.append(runner_data)

    return pd.DataFrame(betfair_data)


In [128]:
def find_arbitrage(polymarket_df, betfair_df):
    import numpy as np
    # Load market and selection mappings
    market_mapping = pd.read_csv('../data/market_mapping.csv')
    selection_mapping = pd.read_csv('../data/selection_mapping.csv')

    # Convert IDs to strings in all DataFrames
    polymarket_df['Market ID'] = polymarket_df['Market ID'].astype(str)
    polymarket_df['Bet ID'] = polymarket_df['Bet ID'].astype(str)
    betfair_df['Market ID'] = betfair_df['Market ID'].astype(str)
    betfair_df['Selection ID'] = betfair_df['Selection ID'].astype(str)
    market_mapping['polymarket_market_id'] = market_mapping['polymarket_market_id'].astype(str)
    market_mapping['betfair_market_id'] = market_mapping['betfair_market_id'].astype(str)
    selection_mapping['polymarket_bet_id'] = selection_mapping['polymarket_bet_id'].astype(str)
    selection_mapping['betfair_selection_id'] = selection_mapping['betfair_selection_id'].astype(str)

    # Apply mappings
    polymarket_df = pd.merge(polymarket_df, market_mapping, left_on='Market ID', right_on='polymarket_market_id', how='left')
    polymarket_df = pd.merge(polymarket_df, selection_mapping, left_on='Bet ID', right_on='polymarket_bet_id', how='left')

    # Merge Polymarket and Betfair data
    merged_df = pd.merge(
        polymarket_df,
        betfair_df,
        left_on=['betfair_market_id', 'betfair_selection_id'],
        right_on=['Market ID', 'Selection ID'],
        how='inner',
        suffixes=('_poly', '_bet')
    )

    arbitrage_opportunities = []
    for _, row in merged_df.iterrows():
        # Calculate implied odds
        if row['Yes Price'] > 0:
            O_poly_yes = 1 / row['Yes Price']
        else:
            O_poly_yes = np.nan

        if row['No Price'] > 0:
            O_poly_no = 1 / row['No Price']
        else:
            O_poly_no = np.nan

        O_bet_back = row['Best Back Price']
        O_bet_lay = row['Best Lay Price']

        # Calculate implied probabilities
        P_poly_yes = row['Yes Price']
        P_poly_no = row['No Price']
        P_bet_yes_back = 1 / O_bet_back if O_bet_back else np.nan
        P_bet_yes_lay = 1 / O_bet_lay if O_bet_lay else np.nan

        # Check for arbitrage opportunities
        # Scenario 1: Back "Yes" on Polymarket, Lay "Yes" on Betfair
        if not np.isnan(O_poly_yes) and not np.isnan(O_bet_lay) and O_poly_yes > O_bet_lay:
            arbitrage_opportunities.append({
                'Market': row['Title'],
                'Selection': row['Selection Name'],
                'Type': 'Back Yes on Polymarket, Lay Yes on Betfair',
                'Polymarket Odds': O_poly_yes,
                'Betfair Lay Odds': O_bet_lay,
                'Profit %': (O_poly_yes / O_bet_lay - 1) * 100
            })

        # Scenario 2: Back No on Polymarket, Back Yes on Betfair
        if not np.isnan(O_poly_no) and not np.isnan(O_bet_back):
            total_prob = P_poly_no + (1 / O_bet_back)
            if total_prob < 1:
                profit_percent = (1 - total_prob) * 100
                arbitrage_opportunities.append({
                    'Market': row['Title'],
                    'Selection': row['Selection Name'],
                    'Type': 'Back No on Polymarket, Back Yes on Betfair',
                    'Polymarket Odds': O_poly_no,
                    'Betfair Back Odds': O_bet_back,
                    'Profit %': profit_percent
                })

    return pd.DataFrame(arbitrage_opportunities)

In [130]:
# Main execution
polymarket_data = get_polymarket_data()
betfair_data = get_betfair_data()
arbitrage_opportunities = find_arbitrage(polymarket_data, betfair_data)

In [131]:
print(arbitrage_opportunities)

                                   Market      Selection  \
0       Presidential Election Winner 2024   Donald Trump   
1       Presidential Election Winner 2024  Kamala Harris   
2   Michigan Presidential Election Winner      Democrats   
3  Wisconsin Presidential Election Winner      Democrats   

                                         Type  Polymarket Odds  \
0  Back No on Polymarket, Back Yes on Betfair         3.025719   
1  Back Yes on Polymarket, Lay Yes on Betfair         3.034901   
2  Back Yes on Polymarket, Lay Yes on Betfair         2.020202   
3  Back Yes on Polymarket, Lay Yes on Betfair         2.409639   

   Betfair Back Odds   Profit %  Betfair Lay Odds  
0               1.57   3.255732               NaN  
1                NaN   9.169114              2.78  
2                NaN  12.233446              1.80  
3                NaN   6.621175              2.26  
